In [ ]:
# |default_exp chroma_db

In [ ]:
# | hide
from nbdev.showdoc import *
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

## Install dependencies

## Make an app with Gradio

In [ ]:
# |export
import chromadb
from chromadb.config import Settings
from dotenv import load_dotenv
import os
import ollama
import numpy as np
import hashlib

This is a simple RAG chatbot built on top of Llama Index and Gradio. It allows you to upload any text or PDF files and ask questions about them!
Before running this, make sure you have exported your OpenAI API key as an environment variable:

```bash
export OPENAI_API_KEY="mykey"
```

In [ ]:
# |export
load_dotenv()

In [ ]:
oai_key = os.getenv("GEMINI_API_KEY")
print(oai_key)

In [ ]:
http_proxy = os.getenv("HTTP_PROXY")
https_proxy = os.getenv("HTTPS_PROXY")
print(http_proxy, https_proxy)

# os.environ["HTTP_PROXY"] = ''
# os.environ["HTTPS_PROXY"] = ''
# http_proxy = os.getenv("HTTP_PROXY")
# https_proxy = os.getenv("HTTPS_PROXY")
# print(http_proxy, https_proxy)

In [ ]:
# |export
# client = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet",persist_directory="db/"))
documents = [
  "Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels",
  "Llamas were first domesticated and used as pack animals 4,000 to 5,000 years ago in the Peruvian highlands",
  "Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall",
  "Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight",
  "Llamas are vegetarians and have very efficient digestive systems",
  "Llamas live to be about 20 years old, though some only live for 15 years and others live to be 30 years old",
]
client = chromadb.PersistentClient(path="../db", settings=Settings(allow_reset=True))
try:
  collection = client.get_collection(name="test_collection")
  client.reset()
  # client.delete_collection(name="test_collection2")
except Exception as e:
  print(e)

collection = client.create_collection(name="test_collection")
client.list_collections()

In [ ]:
# |export
student_info = """
Alexandra Thompson, a 19-year-old computer science sophomore with a 3.7 GPA,
is a member of the programming and chess clubs who enjoys pizza, swimming, and hiking
in her free time in hopes of working at a tech company after graduating from the University of Washington.
"""

club_info = """
The university chess club provides an outlet for students to come together and enjoy playing
the classic strategy game of chess. Members of all skill levels are welcome, from beginners learning
the rules to experienced tournament players. The club typically meets a few times per week to play casual games,
participate in tournaments, analyze famous chess matches, and improve members' skills.
"""

university_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world.
"""

documents.extend([student_info, club_info, university_info])

In [ ]:
for i, d in enumerate(documents):
    response = ollama.embed(model='bge-m3',input=d)
    embeddings = response.embeddings
    print(np.linalg.norm(response.embeddings),response.embeddings)
    collection.add(
        ids=[hashlib.sha256(d.encode()).hexdigest()],
        metadatas=[{"text": d[:15]}],
        embeddings=embeddings,  # type: ignore
        documents=[d],
    )

In [ ]:
# |export
# an example input
# request = "What animals are llamas related to?"
request = "What is the club name?"

# generate an embedding for the input and retrieve the most relevant doc
response = ollama.embed(
  model="bge-m3",
  input=request
)
results = collection.query(
  query_embeddings=response["embeddings"],
  n_results=6
)
# data = results['documents'][0][0]

In [ ]:
results['documents'][0][0] if results['documents'] else None
results['ids'][0][0] if results['ids'] else None
results['metadatas'][0][0] if results['metadatas'] else None

In [ ]:

results['documents'][0][1] if results['documents'] else None
results['ids'][0][1] if results['ids'] else None
results['metadatas'][0][1] if results['metadatas'] else None

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()